In [2]:
import nest_asyncio
# Allows nested asyncio event loops - Jupyter already runs in an event loop,
# so nest_asyncio lets us use asyncio.run() within this environment.
nest_asyncio.apply()

In [1]:
from haystack import Pipeline
from haystack.components.agents import Agent
from haystack_integrations.components.generators.anthropic import AnthropicChatGenerator
from haystack.components.tools import ToolInvoker
from haystack.components.converters import OutputAdapter
from haystack.dataclasses import ChatMessage

In [4]:
import getpass
import os

os.environ['ANTHROPIC_API_KEY'] = getpass.getpass(prompt='Enter your Anthropic API key: ')

Enter your Anthropic API key:  ········


In [3]:
# llm = OllamaChatGenerator(model="qwen2.5:32b", timeout=90, url="http://localhost:11434", streaming_callback=lambda chunk: print(chunk.content, end="", flush=True))

In [ ]:
# messages = [
#     ChatMessage.from_user("What's Natural Language Processing?"),
#     ChatMessage.from_system(
#         "Natural Language Processing (NLP) is a field of computer science and artificial "
#         "intelligence concerned with the interaction between computers and human language"
#     ),
#     ChatMessage.from_user("How do I get started?"),
# ]

# response = llm.run(messages, generation_kwargs={"temperature": 0.2})

# print(response["replies"][0].text)

Natural Language Processing (NLP) is a branch of Artificial Intelligence that focuses on enabling computers to understand, interpret, and generate human languages in a valuable way. It involves the development of algorithms and models that can process large amounts of natural language data to perform tasks such as text classification, sentiment analysis, machine translation, speech recognition, and more.

To get started with NLP, you can follow these steps:

1. **Learn the Basics**: Start by understanding the fundamental concepts in linguistics and computer science relevant to NLP. This

In [5]:
from haystack_integrations.tools.mcp import MCPToolset, StdioServerInfo, SSEServerInfo

In [6]:
server_info = StdioServerInfo(command="uv", args=["run", "--with", "biomcp-python", "biomcp", "run"])

In [9]:
mcp_toolset = MCPToolset(server_info)

In [10]:
mcp_toolset.tools

[Tool(name='article_details', description="\n    Retrieves details for a single PubMed article given its\n    PubMed ID (PMID).\n    Input: A single PMID (e.g., 34397683)\n    Process: Calls the PubTator3 API to fetch the article's\n             title, abstract, and full text (if available).\n    Output: A Markdown formatted string containing the\n            retrieved article content.\n    ", parameters={'properties': {'pmid': {'title': 'pmid', 'type': 'string'}}, 'required': ['pmid'], 'title': 'article_detailsArguments', 'type': 'object'}, function=<function MCPToolset.__init__.<locals>.create_invoke_tool.<locals>.invoke_tool at 0x11430e660>, outputs_to_string=None, inputs_from_state=None, outputs_to_state=None),
 Tool(name='article_searcher', description='\n    Searches PubMed articles using structured criteria.\n    Input: A `PubmedRequest` object containing lists for `genes`,\n           `variants`, `diseases`, `chemicals`, and `keywords`.\n           Use full terms ("Non-small ce

In [8]:
# Create the toolset - this will automatically discover all available tools

pipeline = Pipeline()
pipeline.add_component("llm",  AnthropicChatGenerator(model="claude-3-7-sonnet-20250219", tools=mcp_toolset, streaming_callback=lambda chunk: print(chunk.content, end="", flush=True)))
pipeline.add_component("tool_invoker", ToolInvoker(tools=mcp_toolset))
pipeline.add_component(
    "adapter",
    OutputAdapter(
        template="{{ initial_msg + initial_tool_messages + tool_messages }}",
        output_type=list[ChatMessage],
        unsafe=True,
    ),
)
pipeline.add_component("response_llm",  AnthropicChatGenerator(model="claude-3-7-sonnet-20250219", streaming_callback=lambda chunk: print(chunk.content, end="", flush=True)))
pipeline.connect("llm.replies", "tool_invoker.messages")
pipeline.connect("llm.replies", "adapter.initial_tool_messages")
pipeline.connect("tool_invoker.tool_messages", "adapter.tool_messages")
pipeline.connect("adapter.output", "response_llm.messages")

user_input = "Create a short report on clinical trails with Diclofenac."  # can be any city
user_input_msg = ChatMessage.from_user(text=user_input)

result = pipeline.run({"llm": {"messages": [user_input_msg]}, "adapter": {"initial_msg": [user_input_msg]}})

print(result)

Unsafe mode is enabled. This allows execution of arbitrary code in the Jinja template. Use this only if you trust the source of the template.


I'd be happy to create a short report on clinical trials involving Diclofenac. Let me search for relevant clinical trials using the trial_searcher tool.# Report on Clinical Trials with Diclofenac

## Introduction
Diclofenac is a non-steroidal anti-inflammatory drug (NSAID) that is commonly used to treat pain, inflammation, and stiffness. This report summarizes key findings from clinical trials investigating various forms and applications of diclofenac across different medical conditions.

## Formulations and Delivery Methods
Clinical trials have evaluated multiple diclofenac formulations:
- Oral tablets/capsules (diclofenac sodium, diclofenac potassium)
- Topical gels (including Voltaren Gel, Solaraze, PENNSAID)
- Transdermal patches
- Injectable forms (intramuscular, subcutaneous)
- Rectal suppositories
- Novel delivery systems (trans-mucosal bio-adhesive discs, in-situ gels)

## Major Therapeutic Applications

### Pain Management
Several trials have assessed diclofenac's efficacy for

In [ ]:
result